In [39]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [40]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196'

In [41]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [42]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,kO9EW950IW3GG2BJMM2nZXV0X9XljZh4tmMyBCKI10THSZ4,L_AYRHh0Kbg2yETEfTO0wuPfAPBPhI8yH8tE-qoAlbI-vSg,j9twvE0iouy-mU175H0QaAVjr1hlhR_3IMXz9GZQmxOyXd...,OG Fuzi,3634,1656258378000,508,kO9EW950IW3GG2BJMM2nZXV0X9XljZh4tmMyBCKI10THSZ4,OG Fuzi,509,I,70,31,False,False,False,True
1,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,v0V8za9lUe9Sal3pWMudPbeLKAV9KJam1IhcUktvoCpwFQ,FjLKIVYYUSrV37F_ZcBfkdPfRtkqXlYgw5u8WWv6WI7MYX...,zHydration,585,1656461416000,37,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,TyIer Franklin,969,I,106,52,False,False,True,True
2,24xSh26OwD3onDydygWxFqABPPc4uq35QqOTxk3y-lpYJU...,WykjzKc3LAw7nx_6Yewo8tpj8PQC_Wmg4HxhAduFqfoAvu...,6o8dmWsC8KPRIbdoc_odkL2OXQzVpjfJntTSU6v1rvAmLd...,dis rat hiking,29,1656447596000,1,24xSh26OwD3onDydygWxFqABPPc4uq35QqOTxk3y-lpYJU...,dis rat hiking,612,I,88,55,False,False,True,True
3,rg2dNoXtS1jhecL99tPUbEbv-KX5c55Id8Nzx_jLfzUQ9lY,U7-um4aTa7a9WKtaP5SJo3YKVkvVbj0Y87C1tywb-iHKLw,7Sd2BCrssyyY8zDH-5bopHl6Zj9eqNKj5tsQqBHferw929...,kramsey,5067,1656458938000,214,rg2dNoXtS1jhecL99tPUbEbv-KX5c55Id8Nzx_jLfzUQ9lY,kramsey,443,I,90,59,False,False,True,False
4,NZUyBA6Bjiy_MZj8ShTluVnWxVfPA3NiKmzGuOCH2zNPkhM,sVsaODH09TvWBNVl4uLjnx4rHJ1DeAGAdUFf8LNIMj8jym4,dZKtDFv3NazDhbfzEYQh3Lz67zQfRJFblxCWcI6ZrSh6Ar...,ajmonsta,1591,1656449463000,399,NZUyBA6Bjiy_MZj8ShTluVnWxVfPA3NiKmzGuOCH2zNPkhM,ajmonsta,602,I,126,97,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,3TTmvP2TOYaLd-yGHjTSDrdB4fo9mgHg0Z3347_5mvEyLCE,M8z3qffZkrZoFzz6Nh6yOCHCI1RdH10QbAEO2rlO4nfFWiU,SIxmDzDxdLvM02pCXLiUI86YYgmqCozEeXBst7wGjAfen4...,casparwu,1114,1656457244000,85,3TTmvP2TOYaLd-yGHjTSDrdB4fo9mgHg0Z3347_5mvEyLCE,casparwu,1004,I,168,121,False,False,True,False
81,aj6DzW4oH8RZ2UqQjKgJFU6DUBAs1lJjLALaao5n7pu8w4ld,mNHRtKGrstDC05AIrvrB35ncviF27Qx65RP7QWo2nEY9tO...,bQvzO0jctSWTHc8MDAkVxB9NYOJlPXAPinXg5fgi7qYSSl...,XII Bobo Nieroli,3592,1656049655577,121,aj6DzW4oH8RZ2UqQjKgJFU6DUBAs1lJjLALaao5n7pu8w4ld,XII Bobo Nieroli,509,I,65,27,False,False,True,False
82,O5jsqhLy0mX6raPFx0wo1wnZCTUs86-FL3tDd3_9Nq1TTdUE,ersZ7nn2iDcLYBFW5YrvKW1WXJVne_nwk2u99957Xe-V84...,FEFuaQWa6yJM8q-VhDOh6TopPwhxNg8OCXBc9xi_c0R2Yt...,NG Tae,3542,1656460512000,25,O5jsqhLy0mX6raPFx0wo1wnZCTUs86-FL3tDd3_9Nq1TTdUE,NG Tae,479,I,177,148,False,False,True,False
83,F-DeDuSr06nYbqaZa9iX6-LfQAvqEw0VRq3uOwoR4kGuY74,47tq2-1S4knyiEriVDqXZqe339RrRWfijf6DJVmGVLex7g,7mnGAKJG2De9JA4PYVvQleZCUlVAUK-0hR_lvjRjLDsny0...,Connerisme,503,1656461206843,254,F-DeDuSr06nYbqaZa9iX6-LfQAvqEw0VRq3uOwoR4kGuY74,Connerisme,470,I,102,66,False,False,True,False


In [43]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [44]:
len(uniq_matches_id)

2923

In [45]:
len(matches_asset)

3671

In [46]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [47]:
len(uniq_matches)

2923

In [48]:
matches_asset[0]['info']['game_version']

'Version 12.11.446.9344 (Jun 09 2022/13:23:39) [PUBLIC] <Releases/12.11>'

In [49]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [50]:
len(latest_matches)

1031

In [51]:
# latest_matches[0]['info']

In [52]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [53]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4354923614',
 'placement': 2,
 'augment0': 'TFT7_Augment_AssassinEmblem',
 'augment1': 'TFT7_Augment_BandOfThieves1',
 'augment2': 'TFT6_Augment_CelestialBlessing1',
 'Set7_Assassin': 2,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 0,
 'Set7_Guardian': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Mage': 0,
 'Set7_Ragewing': 0,
 'Set7_Scalescorn': 2,
 'Set7_Tempest': 1,
 'Set7_Warrior': 1,
 'TFT7_Qiyana': 2,
 'TFT7_Qiyana_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Qiyana_item1': 'TFT_Item_TitansResolve',
 'TFT7_Qiyana_item2': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Lillia': 2,
 'TFT7_Shen': 2,
 'TFT7_Braum': 2,
 'TFT7_Braum_item0': 'TFT_Item_WarmogsArmor',
 'TFT7_Braum_item1': 'TFT_Item_Redemption',
 'TFT7_Olaf': 3,
 'TFT7_Olaf_item0': 'TFT_Item_RapidFireCannon',
 'TFT7_Olaf_item1': 'TFT7_Item_AssassinEmblemItem',
 'TFT7_Olaf_item2': 'TFT_Item_InfinityEdge',
 'TFT7_Diana': 3,
 'TFT7_Diana_item0': 'TFT_Item_HextechGunblade',
 'TFT7_Diana_item1': 'TFT_Item_ArchangelsStaff',
 'TFT7_

In [54]:
matches_league_df = pd.json_normalize(matches_array)

In [55]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,Set7_Guild,...,TFT7_Aatrox_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Skarner_item2,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4354923614,2,TFT7_Augment_AssassinEmblem,TFT7_Augment_BandOfThieves1,TFT6_Augment_CelestialBlessing1,2.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4354923614,1,TFT6_Augment_TomeOfTraits1,TFT7_Augment_BandOfThieves1,TFT7_Augment_RevelTrait,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4354923614,5,TFT6_Augment_TrueTwos,TFT6_Augment_Featherweights1,TFT6_Augment_PandorasItems,1.0,1.0,0.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4354923614,6,TFT6_Augment_RichGetRicher,TFT7_Augment_WarriorTrait,TFT6_Augment_ItemGrabBag1,2.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4354923614,3,TFT7_Augment_MageConference,TFT7_Augment_AxiomArc1,TFT6_Augment_ItemGrabBag1,NaN,0.0,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8243,NA1_4356203136,8,TFT7_Augment_AFK,TFT6_Augment_SecondWind1,TFT6_Augment_ThrillOfTheHunt2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8244,NA1_4356203136,7,TFT6_Augment_TinyTitans,TFT6_Augment_CelestialBlessing1,TFT6_Augment_Ascension,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8245,NA1_4356203136,4,TFT6_Augment_Electrocharge1,TFT6_Augment_SecondWind1,TFT7_Augment_RagewingScorch,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8246,NA1_4356203136,1,TFT6_Augment_Weakspot,TFT6_Augment_Diversify1,TFT6_Augment_Diversify2,1.0,NaN,NaN,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,Set7_Guild,...,TFT7_Aatrox_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Skarner_item2,TFT7_Sejuani_item1,TFT7_Sejuani_item2,TFT7_Senna_item1,TFT7_Senna_item2
0,NA1_4354923614,2,TFT7_Augment_AssassinEmblem,TFT7_Augment_BandOfThieves1,TFT6_Augment_CelestialBlessing1,2.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4354923614,1,TFT6_Augment_TomeOfTraits1,TFT7_Augment_BandOfThieves1,TFT7_Augment_RevelTrait,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4354923614,5,TFT6_Augment_TrueTwos,TFT6_Augment_Featherweights1,TFT6_Augment_PandorasItems,1.0,1.0,0.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4354923614,6,TFT6_Augment_RichGetRicher,TFT7_Augment_WarriorTrait,TFT6_Augment_ItemGrabBag1,2.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4354923614,3,TFT7_Augment_MageConference,TFT7_Augment_AxiomArc1,TFT6_Augment_ItemGrabBag1,NaN,0.0,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8243,NA1_4356203136,8,TFT7_Augment_AFK,TFT6_Augment_SecondWind1,TFT6_Augment_ThrillOfTheHunt2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8244,NA1_4356203136,7,TFT6_Augment_TinyTitans,TFT6_Augment_CelestialBlessing1,TFT6_Augment_Ascension,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8245,NA1_4356203136,4,TFT6_Augment_Electrocharge1,TFT6_Augment_SecondWind1,TFT7_Augment_RagewingScorch,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8246,NA1_4356203136,1,TFT6_Augment_Weakspot,TFT6_Augment_Diversify1,TFT6_Augment_Diversify2,1.0,NaN,NaN,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_matches.pickle'))